In [ ]:
# nn.Sequential
#     是一个容器（容器模块），会按照定义的顺序，把多个 nn.Module（层）组合在一起，前向传播时自动按顺序执行

In [1]:
# 用法1:
# nn.Sequential(
#     nn.Conv2d(3, 16, kernel_size=3, padding=1),
#     nn.BatchNorm2d(16),
#     nn.ReLU(),
#     nn.MaxPool2d(2, 2)
# )

# 用法2（推荐）:
# from collections import OrderedDict
# nn.Sequential(OrderedDict([
#     ('conv', nn.Conv2d(3, 16, 3, padding=1)),
#     ('bn', nn.BatchNorm2d(16)),
#     ('relu', nn.ReLU()),
#     ('pool', nn.MaxPool2d(2, 2))
# ]))

# 更推荐用法2的原因：
# 1. 可以通过名字访问每一层，方便调试和修改
    # model['conv1']  # 立刻知道是卷积层
    # model['relu1']  # 清晰知道是哪一层
# 2.可按名字访问/替换层，调试更方便
    # model['relu1'] = nn.LeakyReLU()
# 3.与 state_dict() 更好配合
    # state_dict() 是模型中所有可学习参数（权重和偏置）以及某些缓冲（如 BatchNorm 的 running_mean）的字典表示。
    # 它是 PyTorch 保存/加载模型的核心机制，格式为：
    #     OrderedDict({
    #     'layer_name.parameter_name': tensor
    #     })
    # 在查看模型参数时，使用 OrderedDict 可以更清晰地看到每一层的参数名称和对应的 tensor。
    # from collections import OrderedDict
    # 
    # model = nn.Sequential(OrderedDict([
    #     ('conv1', nn.Conv2d(3, 16, 3)),
    #     ('relu1', nn.ReLU()),
    #     ('conv2', nn.Conv2d(16, 32, 3))
    # ]))
    # 
    # for name, param in model.state_dict().items():
    #     print(name)
    # 上述代码输出如下：
    #     conv1.weight
    #     conv1.bias
    #     conv2.weight
    #     conv2.bias
# 4.支持模型可视化工具
    # 如使用 torchsummary 或 torchviz、TensorBoard 可视化时，命名的层更容易在图中被标记出来，结构层次一目了然。

本次教程使用LeNet作为示例，想了解[LeNet](https://zh.d2l.ai/chapter_convolutional-neural-networks/lenet.html)的可以直接点击链接学习。   
LeNet的结构如下：   
![LeNet结构](./images/Structure-of-LeNet-5.png).  

In [31]:
import torch
import torchvision
from torch import nn
from collections import OrderedDict
from torch.utils.tensorboard import SummaryWriter

In [29]:
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        
        # # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, 
        # #                 groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        # self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 5, padding = 2)
        # # torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)[source]
        # self.maxpool = nn.MaxPool2d(kernel_size = 2)
        # self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 5, padding = 2)
        # self.maxpool2 = nn.MaxPool2d(kernel_size = 2)
        # self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5, padding = 2)
        # self.maxpool3  = nn.MaxPool2d(kernel_size = 2)
        # self.flatten = nn.Flatten()
        # self.linear1 = nn.Linear(1024, 64)
        # self.linear2 = nn.Linear(64, 10)

        # 如此便可以省略掉上面的部分
        # self.model1 = nn.Sequential(
        #     nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 5, padding = 2),
        #     nn.MaxPool2d(kernel_size = 2),
        #     nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 5, padding = 2),
        #     nn.MaxPool2d(kernel_size = 2),
        #     nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5, padding = 2),
        #     nn.MaxPool2d(kernel_size = 2),
        #     nn.Flatten(),
        #     nn.Linear(1024, 64),
        #     nn.Linear(64, 10)
        # )

        # 更推荐这种写法
        self.model1 = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2)),
            ('pool1', nn.MaxPool2d(kernel_size=2)),
            ('conv2', nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2)),
            ('pool2', nn.MaxPool2d(kernel_size=2)),
            ('conv3', nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2)),
            ('pool3', nn.MaxPool2d(kernel_size=2)),
            ('flatten', nn.Flatten()),
            ('fc1', nn.Linear(1024, 64)),  # 注意：1024 = 64通道 × 4 × 4（针对输入32x32）
            ('fc2', nn.Linear(64, 10))
        ]))

    def forward(self, x):
        # x = self.conv1(x)
        # x = self.maxpool(x)
        # x = self.conv2(x)
        # x = self.maxpool2(x)
        # x = self.conv3(x)
        # x = self.maxpool3(x)
        # x = self.flatten(x)
        # x = self.linear1(x)
        # x = self.linear2(x)
        x = self.model1(x)
        return x

In [30]:
tudui = Tudui()
print(tudui)
input = torch.ones((64, 3, 32, 32))
output = tudui(input)
print(output.shape)

Tudui(
  (model1): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc1): Linear(in_features=1024, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=10, bias=True)
  )
)
torch.Size([64, 10])


In [33]:
# 除了print外一个更佳的可视化方式
writer = SummaryWriter("../logs/12_nn_Sequential")
writer.add_graph(tudui, input)
writer.close()

In [ ]:
dataset = torchvision.datasets.CIFAR10("../datasets/CIFAR10", train = False, download = True)
dataloader = torch.utils.dataloader(dataset, batch_size = 32, 